In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q transformers accelerate

In [3]:
!pip install -U bitsandbytes

In [4]:
import torch
import requests
from PIL import Image
from io import BytesIO
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration, BitsAndBytesConfig

model_path = "Qwen/Qwen2-VL-7B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config
)

processor = AutoProcessor.from_pretrained(model_path)

2025-10-31 10:34:47.212328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761906887.235201     418 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761906887.242146     418 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [5]:
# text_prompt="""
# You are an image analysis system. Describe the image with maximum objective detail. Report only what is visible. Do not guess hidden context.

# Required fields:
# 1. Objects:
#    - List all visible objects.
#    - Note size, shape, color, position, and count.

# 2. Environment:
#    - Indoor or outdoor.
#    - Background elements.
#    - Surfaces, textures, lighting, shadows.

# 3. People (if present):
#    - Count.
#    - Visible clothing details.
#    - Actions or poses.
#    - Visible facial expressions strictly from appearance.

# 4. Vehicle details (if present):
#    - Make, model badges, color.
#    - Body style.
#    - Wheel design, lights, grill pattern, mirrors.

# 5. Text/Logos:
#    - Extract any visible text or branding exactly as seen.

# 6. Scene description:
#    - Orientation, camera angle, perspective.
#    - Depth cues, reflections, highlights.

# 7. Visual mood indicators:
#    - Lighting type (harsh, soft, directional).
#    - Color temperature (warm, cool).
#    - Only from observable visual cues.

# 8. Spatial relationships:
#    - Foreground, midground, background positions.
#    - Relative sizes.

# 9. Materials:
#    - Surface finish (matte, glossy, metallic).
#    - Texture patterns.

# Constraints:
# - Do not infer intent, backstory, or emotions beyond visible facial geometry.
# - No opinions or assumptions.
# - Use technical and concise language.
# - Output should be long and highly detailed.
# - Use complete sentences in a single structured paragraph.
# """

In [6]:
# def caption_image_qwen(image_file, prompt):
#     """
#     Generates a caption for a given image file (URL or local path)
#     using the loaded Qwen-VL model and a text prompt.
#     """
#     # Load the image
#     if image_file.startswith('http') or image_file.startswith('https'):
#         response = requests.get(image_file)
#         image = Image.open(BytesIO(response.content)).convert('RGB')
#     else:
#         image = Image.open(image_file).convert('RGB')
        
#     # 1. Create the conversation prompt
#     # Qwen-VL uses a list of dictionaries for the conversation.
#     conversation = [
#         {
#             "role": "user",
#             "content": [
#                 {"type": "image"},
#                 {"type": "text", "text": prompt}
#             ]
#         }
#     ]
    
#     # 2. Apply the chat template and process the inputs
#     # The processor prepares both the text and the image.
#     text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
    
#     # Process the text and image together
#     inputs = processor(
#         text=[text_prompt],  # Note: text is passed as a list
#         images=[image],    # Note: images are passed as a list
#         return_tensors="pt"
#     ).to(model.device)

#     # 3. Generate the caption
#     with torch.inference_mode():
#         output_ids = model.generate(**inputs, max_new_tokens=1024)

#     outputs = processor.batch_decode(output_ids, skip_special_tokens=True)
    
#     # Clean the output
#     cleaned_output = outputs[0].split("\n")[-1].strip()
#     return cleaned_output

In [7]:
# image_url = "https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=small"

# caption = caption_image_qwen(image_url, text_prompt)

# print("--- Generated Caption ---")
# print(caption)

In [8]:
def thumb_extract(url):
  """
  Extracts the actual image URL from the media string.
  e.g., "Photo(thumbnailUrl='http://...')" -> "http://..."
  """
  try:
    return str(url).split("'")[1]
  except Exception:
    return None

In [9]:
def caption_images_batch(image_files_list, prompt):
    """
    Generates captions for a BATCH of image files (URLs or local paths)
    using the loaded Qwen-VL model and a single shared prompt.
    """
    images = []
    max_size = (512, 512) 
    
    for image_file in image_files_list:
        try:
            if image_file.startswith('http') or image_file.startswith('https'):
                response = requests.get(image_file, timeout=10) 
                current_image = Image.open(BytesIO(response.content)).convert('RGB')
            else:
                current_image = Image.open(image_file).convert('RGB')
            
            # --- WHAT I'VE ADDED ---
            # Resize the image in-place while maintaining aspect ratio
            # This makes processing much faster and uses less memory
            current_image.thumbnail(max_size, Image.Resampling.LANCZOS)
            images.append(current_image)
            # --- END OF ADDITION ---
            
        except Exception as e:
            print(f"Error loading image {image_file}: {e}")
            images.append(None) 

    # --- The rest of the function is the same ---
    
    valid_images = [img for img in images if img is not None]
    
    if not valid_images:
        return ["Error: No valid images in batch"] * len(image_files_list)

    conversations = []
    for _ in valid_images:
        conversations.append([
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": prompt}
                ]
            }
        ])

    text_prompts = processor.apply_chat_template(conversations, add_generation_prompt=True, tokenize=False)
    
    inputs = processor(
        text=text_prompts,
        images=valid_images,
        return_tensors="pt",
        padding=True 
    ).to(model.device)

    with torch.inference_mode():
        output_ids = model.generate(
            **inputs, 
            max_new_tokens=512,
            do_sample=False
        )

    input_token_len = inputs['input_ids'].shape[1]
    generated_ids = output_ids[:, input_token_len:]
    responses = processor.batch_decode(generated_ids, skip_special_tokens=True)
    
    cleaned_responses = [resp.strip() for resp in responses]
    
    final_output = []
    resp_index = 0
    for img in images:
        if img is None:
            final_output.append("Error: Image load failed")
        else:
            final_output.append(cleaned_responses[resp_index])
            resp_index += 1
            
    return final_output

In [ ]:
import pandas as pd
import time
import math

BATCH_SIZE = 32
YOUR_CSV_FILE = '/kaggle/input/behaviour-simulation/content_simulation_train.csv'
OUTPUT_CSV_FILE = '/kaggle/working/generated_captions.csv'
MEDIA_COLUMN_NAME = 'media'
NO_MEDIA_STRING = "no media found"

text_prompt="""
Describe everything visible in the image with maximum objective detail, including all objects, environment, people, text, vehicle features, materials, lighting, spatial layout, textures, and reflections without guessing unseen context or intent"""

print("Loading dataset from {YOUR_CSV_FILE}...")
try:
    df = pd.read_csv(YOUR_CSV_FILE)
except FileNotFoundError:
    print(f"Error: File not found at {YOUR_CSV_FILE}")

df['generated_caption'] = None
print(f"Dataset loaded. Total rows: {len(df)}")

total_batches = math.ceil(len(df) / BATCH_SIZE)
total_start_time = time.time()

print(f"Starting processing in batches of {BATCH_SIZE}...")

for i in range(0, len(df), BATCH_SIZE):
    batch_start_time = time.time()
    
    batch_df = df.iloc[i : i + BATCH_SIZE]
    
    urls_to_process = []
    valid_indices = []

    for index, row in batch_df.iterrows():
        media_data = str(row["media"])
        
        if NO_MEDIA_STRING not in media_data.lower() and pd.notna(media_data):
            extracted_url = thumb_extract(media_data)
            if extracted_url:
                urls_to_process.append(extracted_url)
                valid_indices.append(index) # यह इंडेक्स (e.g., 20, 21, 23, 27...)
            else:
                df.at[index, 'generated_caption'] = "Error: URL extraction failed"
        else:
            df.at[index, 'generated_caption'] = ""

    if urls_to_process:
        print(f"Processing Batch {i//BATCH_SIZE + 1}/{total_batches} (Found {len(urls_to_process)} valid images)...")
        
        batch_captions = caption_images_batch(urls_to_process, text_prompt)
        
        for j, caption in enumerate(batch_captions):
            original_index = valid_indices[j]
            df.at[original_index, 'generated_caption'] = caption
    else:
        print(f"Skipping Batch {i//BATCH_SIZE + 1}/{total_batches} (No valid images found)")

    batch_end_time = time.time()
    print(f"Batch {i//BATCH_SIZE + 1} finished in {batch_end_time - batch_start_time:.2f} seconds.")

    if (i // BATCH_SIZE) % 5 == 0:
        print(f"Saving intermediate results to {OUTPUT_CSV_FILE}...")
        df.to_csv(OUTPUT_CSV_FILE, index=False)


total_end_time = time.time()
print("\n--- Processing Complete ---")
print(f"Total time taken: {(total_end_time - total_start_time) / 60:.2f} minutes.")
print(f"Saving final results to {OUTPUT_CSV_FILE}...")
df.to_csv(OUTPUT_CSV_FILE, index=False)

print("Done.")

Loading dataset from {YOUR_CSV_FILE}...
